## TP3

Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en
Material_TPs/TP3/images a partir del template Material_TPs/TP3/template
1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos posit. os
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo témplate del ítem 1
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección

In [97]:
%matplotlib qt

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import glob
from sklearn.cluster import DBSCAN

## Punto 1

### Vamos a probar con los 6 métodos de detección de patrones y observar en donde obtenemos falsos positivos para descartarlos. 
#### Comenzamos con la imagen logo_1.pngng"

In [77]:
# Cargamos la imagen a procesar
img = cv.imread('images/logo_1.png')

img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.imread('template/pattern.png',0)

plt.figure()
plt.subplot(1,2,1)
plt.imshow(template, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_rgb)

In [78]:
w, h = template.shape[::-1]
# Los 6 métodos posibles para comparación:
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

for meth in methods:
    # Hago una copia de la imagen porque ciclo a ciclo le dibujo rectángulos
    img_salida = img_rgb.copy()
    
    method = eval(meth)
    
    # Aplicamos la coincidencia de patrones
    #--------------------------------------
    res = cv.matchTemplate(img_gray, template, method)
    
    # Encontramos los valores máximos y mínimos
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    
    # Si el método es TM_SQDIFF o TM_SQDIFF_NORMED, tomamos el mínimo
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    
    # Marcamos el lugar donde lo haya encontrado
    #----------------------------------------
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img_salida,top_left, bottom_right, 255, 2)
    
    # Graficamos el procesamiento y la salida
    #----------------------------------------
    plt.figure()
    
    # Resultado de coincidencia
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    
    # Imagen original con recuadros
    plt.subplot(122),plt.imshow(img_salida)
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    
    plt.suptitle(meth)
    plt.show()

Podemos observar lo siguiente:
1. Con el metodo **Coeficiente de correlación cruzada** obtenemos un falso positivo parcial ya que al parecer detecta parte del  logo pero no llega a reconocerlo completamente.
2. Con el metodo **Coeficiente de correlación cruzada normalizado** observamos el mismo comportamiento que con TM_CCOEFF.
3. Con el metodo de **Correlación cruzada** observamos que tenemos una correcta deteccion salvo por una pequeña parte del logo que no llega a detectar.
4. Con el metodo de **Correlación cruzada normalizada** al igual que en TM_CCOEFF y TM_CCOEFF_NORMED observamos que solo detecta una parte del logo.
5. Con el metodo de **CDiferencia cuadrática media (SSD)** observamos que si hace una correcta detección del logo encerrando en el cuadro gran parte del logo pero dejando afuera una parte de la primera "C" de la palabra Coca Cola.
6. Con el metodo de **CDiferencia cuadrática media (SSD) normalizado** mismo resultado que con TM_SQDIFF

#### Hacemos lo mismo para "coca_logo_1".

In [79]:
# Cargamos la imagen a procesar
img = cv.imread('images/coca_logo_1.png')

img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.imread('template/pattern.png',0)

plt.figure()
plt.subplot(1,2,1)
plt.imshow(template, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_rgb)

In [80]:
template_scaled = cv.resize(template, (0, 0), fx=0.5, fy=0.7, interpolation=cv.INTER_AREA)

En el caso de esta imagen el codigo me da un error ya que la imagen es más pequeña que el template, lo que hago es primero hacer una detección de dicho problema para cada metodo e implemento un metodo para escalar la imagen del tamaño del template para poder implementar los metodos

In [81]:
def escalar_imagen_si_necesario(img_rgb, img_gray, template):
    h_img, w_img = img_gray.shape
    h_temp, w_temp = template.shape

    # Si la imagen es más pequeña que el template en cualquier dimensión
    if h_img < h_temp or w_img < w_temp:
        # Calculamos el factor necesario para igualar dimensiones
        scale_h = h_temp / h_img
        scale_w = w_temp / w_img
        scale_factor = max(scale_h, scale_w) + 0.05  # Un poco más para asegurar

        new_width = int(w_img * scale_factor)
        new_height = int(h_img * scale_factor)

        # Escalar ambas imágenes
        img_gray_scaled = cv.resize(img_gray, (new_width, new_height), interpolation=cv.INTER_LINEAR)
        img_rgb_scaled = cv.resize(img_rgb, (new_width, new_height), interpolation=cv.INTER_LINEAR)

        print(f"🔁 Imagen escalada automáticamente a: {new_width}x{new_height}")
        return img_rgb_scaled, img_gray_scaled
    else:
        return img_rgb, img_gray

In [82]:
w, h = template_scaled.shape[::-1]

# Los 6 métodos posibles para comparación:
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
           'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

img_rgb, img_gray = escalar_imagen_si_necesario(img_rgb, img_gray, template_scaled)

for meth in methods:
    img_salida = img_rgb.copy()
    method = eval(meth)

    # Verificación de tamaño
    if img_gray.shape[0] < template_scaled.shape[0] or img_gray.shape[1] < template_scaled.shape[1]:
        print(f"⚠️ El template es más grande que la imagen. Saltando método: {meth}")
        continue

    # Aplicamos la coincidencia de patrones
    res = cv.matchTemplate(img_gray, template_scaled, method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

    # Si el método es SQDIFF, buscamos el mínimo; en otros, el máximo
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc

    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img_salida, top_left, bottom_right, 255, 2)

    # Mostrar resultados
    plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.imshow(res, cmap='gray')
    plt.title('Matching Result')
    plt.xticks([]), plt.yticks([])

    plt.subplot(122)
    plt.imshow(img_salida)
    plt.title('Detected Point')
    plt.xticks([]), plt.yticks([])

    plt.suptitle(meth)
    plt.show()



In [83]:
# Cargar imagen y template originales
img_rgb = cv.imread('images/coca_logo_1.png') 
img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
template_orig = cv.imread('template/pattern.png', 0)  

# Métodos de OpenCV para comparar
methods = {
    'TM_CCOEFF': cv.TM_CCOEFF,
    'TM_CCOEFF_NORMED': cv.TM_CCOEFF_NORMED,
    'TM_CCORR': cv.TM_CCORR,
    'TM_CCORR_NORMED': cv.TM_CCORR_NORMED,
    'TM_SQDIFF': cv.TM_SQDIFF,
    'TM_SQDIFF_NORMED': cv.TM_SQDIFF_NORMED,
}

# Escalas a probar (ajustables)
scales = np.linspace(0.4, 0.47, 64)

# Preparar gráficos
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Procesar cada método
for i, (name, method) in enumerate(methods.items()):
    best_val = -np.inf
    best_match = None
    best_template = None
    best_scale = None

    for scale in scales:
        template_scaled = cv.resize(template_orig, (0, 0), fx=scale, fy=scale, interpolation=cv.INTER_AREA)
        h, w = template_scaled.shape

        if img_gray.shape[0] < h or img_gray.shape[1] < w:
            continue

        res = cv.matchTemplate(img_gray, template_scaled, method)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

        val = -min_val if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED] else max_val

        if val > best_val:
            best_val = val
            best_match = min_loc if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED] else max_loc
            best_template = template_scaled.copy()
            best_scale = scale

    # Dibujar el mejor match
    result_img = img_rgb.copy()
    h, w = best_template.shape
    top_left = best_match
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(result_img, top_left, bottom_right, (0, 255, 0), 2)

    axes[i].imshow(cv.cvtColor(result_img, cv.COLOR_BGR2RGB))
    axes[i].set_title(f"{name} (escala {best_scale:.2f})")
    axes[i].axis('off')

plt.tight_layout()
plt.suptitle("Mejor coincidencia por método (multiescala)", fontsize=18, y=1.05)
plt.show()


- Nos dimos cuenta que todos los metodos nos dan con esta imagen un falso positivo, al detectar zonas con mucho blanco, por lo tanto lo que hacemos es pre-procesar el template y la imagen para poder reducir la informacion a correlacionar, en este caso usamos Canny para ese pre-procesamiento. 

In [84]:
# Cargar imagen y template
img_color = cv.imread('images/coca_logo_1.png')  
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
template_gray = cv.imread('template/pattern.png', 0) 

# Aplicar Canny a la imagen
img_edges = cv.Canny(img_gray, 50, 150)

# Escalar el template
template_scaled = cv.resize(template_gray, (0, 0), fx=0.5, fy=0.5, interpolation=cv.INTER_AREA)
template_edges = cv.Canny(template_scaled, 50, 150)
h, w = template_edges.shape

# Métodos a probar
methods = {
    'TM_CCOEFF': cv.TM_CCOEFF,
    'TM_CCOEFF_NORMED': cv.TM_CCOEFF_NORMED,
    'TM_CCORR': cv.TM_CCORR,
    'TM_CCORR_NORMED': cv.TM_CCORR_NORMED,
    'TM_SQDIFF': cv.TM_SQDIFF,
    'TM_SQDIFF_NORMED': cv.TM_SQDIFF_NORMED
}

# Preparar figura
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Verificar que el template quepa
if img_edges.shape[0] >= h and img_edges.shape[1] >= w:
    for i, (name, method) in enumerate(methods.items()):
        img_result = img_color.copy()
        result = cv.matchTemplate(img_edges, template_edges, method)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        # Según el método, usar mínimo o máximo
        top_left = min_loc if 'SQDIFF' in name else max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)

        cv.rectangle(img_result, top_left, bottom_right, (0, 255, 0), 2)

        axes[i].imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
        axes[i].set_title(f'{name}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.suptitle("Detección del logo con Canny y 6 métodos", fontsize=18, y=1.02)

    # Mostrar template escalado al lado
    plt.figure(figsize=(4, 4))
    plt.imshow(template_edges, cmap='gray')
    plt.title("Template (Canny + Escalado)")
    plt.axis('off')
    plt.show()

else:
    print("El template aún es más grande que la imagen.")

- Finalmente observamos las imagenes pre-procesadas y la deteccion del logo con el metodo TM_CCOEFF_NORMED.

#### Trabajamos lo mismo para "coca_logo_2".

In [85]:
# Cargamos la imagen a procesar
img = cv.imread('images/coca_logo_2.png')

img_rgb= cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_gray= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.imread('template/pattern.png',0)

plt.figure()
plt.subplot(1,2,1)
plt.imshow(template, cmap='gray')
plt.subplot(1,2,2)
plt.imshow(img_rgb)

- Nuevamente obtenemos un error de escalas en donde el template es mas grande que la imagen en donde debemos detectar el logo, por lo que intentamos nuevamente con Canny pero esta vez probamos varias escalas de template para poder hacer la deteccion. 

In [86]:
# Cargar imagen y template
img_color = cv.imread('images/coca_logo_2.png') 
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
template_gray = cv.imread('template/pattern.png', 0) 

# Aplicar Canny a la imagen
img_edges = cv.Canny(img_gray, 50, 150)

# Escalar el template
template_scaled = cv.resize(template_gray, (0, 0), fx=0.5, fy=0.7, interpolation=cv.INTER_AREA)
template_edges = cv.Canny(template_scaled, 50, 150)
h, w = template_edges.shape

# Métodos a probar
methods = {
    'TM_CCOEFF': cv.TM_CCOEFF,
    'TM_CCOEFF_NORMED': cv.TM_CCOEFF_NORMED,
    'TM_CCORR': cv.TM_CCORR,
    'TM_CCORR_NORMED': cv.TM_CCORR_NORMED,
    'TM_SQDIFF': cv.TM_SQDIFF,
    'TM_SQDIFF_NORMED': cv.TM_SQDIFF_NORMED
}

# Preparar figura
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Verificar que el template quepa
if img_edges.shape[0] >= h and img_edges.shape[1] >= w:
    for i, (name, method) in enumerate(methods.items()):
        img_result = img_color.copy()
        result = cv.matchTemplate(img_edges, template_edges, method)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        # Según el método, usar mínimo o máximo
        top_left = min_loc if 'SQDIFF' in name else max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)

        cv.rectangle(img_result, top_left, bottom_right, (0, 255, 0), 2)

        axes[i].imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
        axes[i].set_title(f'{name}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.suptitle("Detección del logo con Canny y 6 métodos", fontsize=18, y=1.02)

    # Mostrar template escalado al lado
    plt.figure(figsize=(4, 4))
    plt.imshow(template_edges, cmap='gray')
    plt.title("Template (Canny + Escalado)")
    plt.axis('off')
    plt.show()

else:
    print("El template aún es más grande que la imagen.")

#### Finalmente hacemos lo mismo con el resto de las imagenes

#### Probamos con la imagen coca_multi.png, que tiene el logo mas pequeño, por lo que  intentamos escalar el template para que sea mas pequeño.

In [87]:
# Cargar imagen y template
img_color = cv.imread('images/coca_multi.png')  
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
template_gray = cv.imread('template/pattern.png', 0)  

# Aplicar Canny a la imagen
img_edges = cv.Canny(img_gray, 50, 150)

# Escalar el template
template_scaled = cv.resize(template_gray, (0, 0), fx=0.2, fy=0.2, interpolation=cv.INTER_AREA)
template_edges = cv.Canny(template_scaled, 50, 150)
h, w = template_edges.shape

# Métodos a probar
methods = {
    'TM_CCOEFF': cv.TM_CCOEFF,
    'TM_CCOEFF_NORMED': cv.TM_CCOEFF_NORMED,
    'TM_CCORR': cv.TM_CCORR,
    'TM_CCORR_NORMED': cv.TM_CCORR_NORMED,
    'TM_SQDIFF': cv.TM_SQDIFF,
    'TM_SQDIFF_NORMED': cv.TM_SQDIFF_NORMED
}

# Preparar figura
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Verificar que el template quepa
if img_edges.shape[0] >= h and img_edges.shape[1] >= w:
    for i, (name, method) in enumerate(methods.items()):
        img_result = img_color.copy()
        result = cv.matchTemplate(img_edges, template_edges, method)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        # Según el método, usar mínimo o máximo
        top_left = min_loc if 'SQDIFF' in name else max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)

        cv.rectangle(img_result, top_left, bottom_right, (0, 255, 0), 2)

        axes[i].imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
        axes[i].set_title(f'{name}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.suptitle("Detección del logo con Canny y 6 métodos", fontsize=18, y=1.02)

    # Mostrar template escalado al lado
    plt.figure(figsize=(4, 4))
    plt.imshow(template_edges, cmap='gray')
    plt.title("Template (Canny + Escalado)")
    plt.axis('off')
    plt.show()

else:
    print("El template aún es más grande que la imagen.")


C:\Users\Lucas.Fajardo\AppData\Local\Temp\ipykernel_26408\1072289646.py:25: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(2, 3, figsize=(18, 10))


#### Probamos con la imagen coca_retro_1.png, que tiene el logo mas grand, por lo que  intentamos escalar el template para que sea mas grande probando diferentes valores de fx y fy.

In [88]:
# Cargar imagen y template
img_color = cv.imread('images/coca_retro_1.png')  
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
template_gray = cv.imread('template/pattern.png', 0)  

# Aplicar Canny a la imagen
img_edges = cv.Canny(img_gray, 50, 150)

# Escalar el template
template_scaled = cv.resize(template_gray, (0, 0), fx=1.3, fy=1.4, interpolation=cv.INTER_AREA)
template_edges = cv.Canny(template_scaled, 50, 150)
h, w = template_edges.shape

# Métodos a probar
methods = {
    'TM_CCOEFF': cv.TM_CCOEFF,
    'TM_CCOEFF_NORMED': cv.TM_CCOEFF_NORMED,
    'TM_CCORR': cv.TM_CCORR,
    'TM_CCORR_NORMED': cv.TM_CCORR_NORMED,
    'TM_SQDIFF': cv.TM_SQDIFF,
    'TM_SQDIFF_NORMED': cv.TM_SQDIFF_NORMED
}

# Preparar figura
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Verificar que el template quepa
if img_edges.shape[0] >= h and img_edges.shape[1] >= w:
    for i, (name, method) in enumerate(methods.items()):
        img_result = img_color.copy()
        result = cv.matchTemplate(img_edges, template_edges, method)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        # Según el método, usar mínimo o máximo
        top_left = min_loc if 'SQDIFF' in name else max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)

        cv.rectangle(img_result, top_left, bottom_right, (0, 255, 0), 2)

        axes[i].imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
        axes[i].set_title(f'{name}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.suptitle("Detección del logo con Canny y 6 métodos", fontsize=18, y=1.02)

    # Mostrar template escalado al lado
    plt.figure(figsize=(4, 4))
    plt.imshow(template_edges, cmap='gray')
    plt.title("Template (Canny + Escalado)")
    plt.axis('off')
    plt.show()

else:
    print("El template aún es más grande que la imagen.")


#### Probamos con la imagen coca_retro_2.png, volvemos a valores de fx y fy de 0.4 y logramos el match con 4 metodos

In [89]:
# Cargar imagen y template
img_color = cv.imread('images/coca_retro_2.png')  
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
template_gray = cv.imread('template/pattern.png', 0)  

# Aplicar Canny a la imagen
img_edges = cv.Canny(img_gray, 50, 150)

# Escalar el template
template_scaled = cv.resize(template_gray, (0, 0), fx=0.4, fy=0.4, interpolation=cv.INTER_AREA)
template_edges = cv.Canny(template_scaled, 50, 150)
h, w = template_edges.shape

# Métodos a probar
methods = {
    'TM_CCOEFF': cv.TM_CCOEFF,
    'TM_CCOEFF_NORMED': cv.TM_CCOEFF_NORMED,
    'TM_CCORR': cv.TM_CCORR,
    'TM_CCORR_NORMED': cv.TM_CCORR_NORMED,
    'TM_SQDIFF': cv.TM_SQDIFF,
    'TM_SQDIFF_NORMED': cv.TM_SQDIFF_NORMED
}

# Preparar figura
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Verificar que el template quepa
if img_edges.shape[0] >= h and img_edges.shape[1] >= w:
    for i, (name, method) in enumerate(methods.items()):
        img_result = img_color.copy()
        result = cv.matchTemplate(img_edges, template_edges, method)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        # Según el método, usar mínimo o máximo
        top_left = min_loc if 'SQDIFF' in name else max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)

        cv.rectangle(img_result, top_left, bottom_right, (0, 255, 0), 2)

        axes[i].imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
        axes[i].set_title(f'{name}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.suptitle("Detección del logo con Canny y 6 métodos", fontsize=18, y=1.02)

    # Mostrar template escalado al lado
    plt.figure(figsize=(4, 4))
    plt.imshow(template_edges, cmap='gray')
    plt.title("Template (Canny + Escalado)")
    plt.axis('off')
    plt.show()

else:
    print("El template aún es más grande que la imagen.")


#### Finalmente probamos con la imagen COCA-COLA-LOGO.jpg, intentamos a valores de fx y fy de 0.4 y logramos el match con 4 metodos

## Punto 2

### CCOEFF_NORMED con Canny

In [90]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = boxes.astype("float")
    pick = []

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)

        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_coca_cola_bottles(img_path, template_path):
    # Cargar las imágenes
    img = cv.imread(img_path)
    template = cv.imread(template_path)
    
    # Convertir a escala de grises
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    
    # Aplicar detección de bordes Canny al patrón
    template_edges = cv.Canny(template_gray, 50, 150)
    
    # Escalar el patrón a un tamaño fijo (0.2 del original)
    scale_width = 0.2
    scale_height = 0.2
    width = int(template_edges.shape[1] * scale_width)
    height = int(template_edges.shape[0] * scale_height)
    resized_template = cv.resize(template_edges, (width, height))
    
    # Aplicar Canny a la imagen de entrada
    img_edges = cv.Canny(img_gray, 150, 150)
    
    # Obtener dimensiones del template
    h, w = resized_template.shape
    
    # Aplicar template matching
    result = cv.matchTemplate(img_edges, resized_template, cv.TM_CCORR_NORMED)
    
    # Umbral de detección
    threshold = 0.22
    locations = np.where(result >= threshold)
    
    # Crear lista de cajas para aplicar NMS
    rects = []
    for pt in zip(*locations[::-1]):
        rects.append([pt[0], pt[1], pt[0] + w, pt[1] + h])
    
    rects = np.array(rects)
    final_boxes = non_max_suppression_fast(rects, 0.3)  # umbral de solapamiento
    
    # Dibujar cajas resultantes
    img_result = img.copy()
    for (x1, y1, x2, y2) in final_boxes:
        cv.rectangle(img_result, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Mostrar resultado
    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title(f"Método: TM_CCORR_NORMED con Canny\nDetecciones únicas: {len(final_boxes)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    return len(final_boxes), img_result

detections_count, result_img = detect_coca_cola_bottles('images/coca_multi.png', 'template/pattern.png')
print(f"Se detectaron {detections_count} botellas de Coca-Cola")

Se detectaron 8 botellas de Coca-Cola


## Punto 3

#### 3. Generalizar el algoritmo del item 2 para todas las imágenes. Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza de la detección

- **Hacemos lo mismo con la imagen cola_retro_2.png logramos detectar el logo cambiando la escala del patron**

In [91]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = boxes.astype("float")
    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)

        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_coca_cola_bottles(img_path, template_path):
    img = cv.imread(img_path)
    template = cv.imread(template_path)

    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)

    template_edges = cv.Canny(template_gray, 50, 150)

    scale_width = 0.35
    scale_height = 0.35
    width = int(template_edges.shape[1] * scale_width)
    height = int(template_edges.shape[0] * scale_height)
    resized_template = cv.resize(template_edges, (width, height))

    img_edges = cv.Canny(img_gray, 200, 200)

    h, w = resized_template.shape
    result = cv.matchTemplate(img_edges, resized_template, cv.TM_CCORR_NORMED)

    threshold = 0.2
    locations = np.where(result >= threshold)

    rects = []
    confidencias = []

    for pt in zip(*locations[::-1]):
        x, y = pt
        rects.append([x, y, x + w, y + h])
        confidencias.append(result[y, x])  # confianza de esa detección

    rects = np.array(rects)
    confidencias = np.array(confidencias)
    final_boxes = non_max_suppression_fast(rects, 0.3)

    img_result = img.copy()

    for i, (x1, y1, x2, y2) in enumerate(final_boxes):
        confidence = result[y1, x1]
        cv.rectangle(img_result, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Mostrar porcentaje de confianza fuera del recuadro (arriba a la izquierda)
        label = f"{confidence * 100:.1f}%"
        text_size = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 20 else y1 + 30  # evita que quede fuera de la imagen

        # Dibujar borde blanco (grosor mayor)
        cv.putText(img_result, label, (text_x, text_y),
           cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 4)

        # Dibujar texto negro encima
        cv.putText(img_result, label, (text_x, text_y),
           cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        print(f"Detección {i+1} → coordenadas: ({x1},{y1}) confianza: {confidence:.2f}")


    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title(f"Detecciones únicas: {len(final_boxes)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return len(final_boxes), img_result

# Ejecutar
detections_count, result_img = detect_coca_cola_bottles('images/coca_retro_2.png', 'template/pattern.png')
print(f"Se detectaron {detections_count} botellas de Coca-Cola")


Detección 1 → coordenadas: (69,192) confianza: 0.20
Se detectaron 1 botellas de Coca-Cola


- **Ahora con la imagen coca_logo_1.png hacemos lo mismo ajustando un poco tambien la escala del patron**

In [92]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = boxes.astype("float")
    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)

        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_coca_cola_bottles(img_path, template_path):
    img = cv.imread(img_path)
    template = cv.imread(template_path)

    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)

    template_edges = cv.Canny(template_gray, 50, 150)

    scale_width = 0.36
    scale_height = 0.36
    width = int(template_edges.shape[1] * scale_width)
    height = int(template_edges.shape[0] * scale_height)
    resized_template = cv.resize(template_edges, (width, height))

    img_edges = cv.Canny(img_gray, 200, 200)

    h, w = resized_template.shape
    result = cv.matchTemplate(img_edges, resized_template, cv.TM_CCORR_NORMED)

    threshold = 0.2
    locations = np.where(result >= threshold)

    rects = []
    scores = []

    for pt in zip(*locations[::-1]):
        x, y = pt
        rects.append([x, y, x + w, y + h])
        scores.append(result[y, x])  # confianza de esta detección

    rects = np.array(rects)
    scores = np.array(scores)
    final_boxes = non_max_suppression_fast(rects, 0.3)

    img_result = img.copy()
    for i, (x1, y1, x2, y2) in enumerate(final_boxes):
        confidence = result[y1, x1]

        # Dibujar el recuadro verde
        cv.rectangle(img_result, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Mostrar el score en porcentaje por fuera del recuadro
        label = f"{confidence * 100:.1f}%"
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 20 else y1 + 30

        # Borde blanco
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 4)

        # Texto negro encima
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        print(f"Detección {i+1} → ({x1},{y1}) confianza: {confidence:.2f}")

    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title(f"Método: TM_CCORR_NORMED con Canny\nDetecciones únicas: {len(final_boxes)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return len(final_boxes), img_result

# Ejecutar
detections_count, result_img = detect_coca_cola_bottles('images/coca_logo_1.png', 'template/pattern.png')
print(f"Se detectaron {detections_count} botellas de Coca-Cola")


Detección 1 → (46,207) confianza: 0.20
Se detectaron 1 botellas de Coca-Cola


- **Con la imagen cola_logo_2.png cambiamos la escala del patron y el umbral de detección**

In [93]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = boxes.astype("float")
    pick = []

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)

        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_coca_cola_bottles(img_path, template_path):
    img = cv.imread(img_path)
    template = cv.imread(template_path)

    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)

    template_edges = cv.Canny(template_gray, 50, 150)

    scale_width = 0.5
    scale_height = 0.5
    width = int(template_edges.shape[1] * scale_width)
    height = int(template_edges.shape[0] * scale_height)
    resized_template = cv.resize(template_edges, (width, height))

    img_edges = cv.Canny(img_gray, 200, 200)

    h, w = resized_template.shape
    result = cv.matchTemplate(img_edges, resized_template, cv.TM_CCORR_NORMED)

    threshold = 0.19
    locations = np.where(result >= threshold)

    rects = []
    for pt in zip(*locations[::-1]):
        x, y = pt
        rects.append([x, y, x + w, y + h])

    rects = np.array(rects)
    final_boxes = non_max_suppression_fast(rects, 0.3)

    img_result = img.copy()
    for i, (x1, y1, x2, y2) in enumerate(final_boxes):
        confidence = result[y1, x1]

        # Dibujar recuadro
        cv.rectangle(img_result, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Calcular posición del texto
        label = f"{confidence * 100:.1f}%"
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 20 else y1 + 30

        # Contorno blanco
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 4)

        # Texto negro
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        print(f"Detección {i+1} → coordenadas: ({x1},{y1}) confianza: {confidence:.2f}")

    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title(f"Método: TM_CCORR_NORMED con Canny\nDetecciones únicas: {len(final_boxes)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return len(final_boxes), img_result

# Ejecutar
detections_count, result_img = detect_coca_cola_bottles('images/coca_logo_2.png', 'template/pattern.png')
print(f"Se detectaron {detections_count} botellas de Coca-Cola")


Detección 1 → coordenadas: (15,115) confianza: 0.19
Se detectaron 1 botellas de Coca-Cola


- **Con logo_1.png cambiamos la escala y tambien ajustamos el umbral de detección**

In [94]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = boxes.astype("float")
    pick = []

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)

        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_coca_cola_bottles(img_path, template_path):
    img = cv.imread(img_path)
    template = cv.imread(template_path)
    
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    
    template_edges = cv.Canny(template_gray, 50, 150)

    scale_width = 0.65
    scale_height = 0.65
    width = int(template_edges.shape[1] * scale_width)
    height = int(template_edges.shape[0] * scale_height)
    resized_template = cv.resize(template_edges, (width, height))

    img_edges = cv.Canny(img_gray, 200, 200)

    h, w = resized_template.shape
    result = cv.matchTemplate(img_edges, resized_template, cv.TM_CCORR_NORMED)

    threshold = 0.15
    locations = np.where(result >= threshold)

    rects = []
    for pt in zip(*locations[::-1]):
        rects.append([pt[0], pt[1], pt[0] + w, pt[1] + h])

    rects = np.array(rects)
    final_boxes = non_max_suppression_fast(rects, 0.3)

    img_result = img.copy()
    for i, (x1, y1, x2, y2) in enumerate(final_boxes):
        confidence = result[y1, x1]

        # Dibujar recuadro verde
        cv.rectangle(img_result, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Preparar etiqueta de porcentaje
        label = f"{confidence * 100:.1f}%"
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 20 else y1 + 30

        # Contorno blanco
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 4)

        # Texto negro encima
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        print(f"Detección {i+1} → coordenadas: ({x1},{y1}) confianza: {confidence:.2f}")

    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title(f"Método: TM_CCORR_NORMED con Canny\nDetecciones únicas: {len(final_boxes)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return len(final_boxes), img_result

# Ejecutar
detections_count, result_img = detect_coca_cola_bottles('images/logo_1.png', 'template/pattern.png')
print(f"Se detectaron {detections_count} botellas de Coca-Cola")


Detección 1 → coordenadas: (191,210) confianza: 0.16
Se detectaron 1 botellas de Coca-Cola


- **Por ultimo con la imagen coca_retro_1.png** 

In [95]:
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    boxes = boxes.astype("float")
    pick = []

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = idxs[-1]
        pick.append(last)

        xx1 = np.maximum(x1[last], x1[idxs[:-1]])
        yy1 = np.maximum(y1[last], y1[idxs[:-1]])
        xx2 = np.minimum(x2[last], x2[idxs[:-1]])
        yy2 = np.minimum(y2[last], y2[idxs[:-1]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:-1]]
        idxs = np.delete(idxs, np.concatenate(([len(idxs) - 1], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def detect_coca_cola_bottles(img_path, template_path):
    img = cv.imread(img_path)
    template = cv.imread(template_path)
    
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    template_gray = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    
    template_edges = cv.Canny(template_gray, 100, 150)

    scale_width = 0.9
    scale_height = 0.9
    width = int(template_edges.shape[1] * scale_width)
    height = int(template_edges.shape[0] * scale_height)
    resized_template = cv.resize(template_edges, (width, height))

    img_edges = cv.Canny(img_gray, 250, 250)

    h, w = resized_template.shape
    result = cv.matchTemplate(img_edges, resized_template, cv.TM_CCORR_NORMED)

    threshold = 0.17
    locations = np.where(result >= threshold)

    rects = []
    for pt in zip(*locations[::-1]):
        rects.append([pt[0], pt[1], pt[0] + w, pt[1] + h])

    rects = np.array(rects)
    final_boxes = non_max_suppression_fast(rects, 0.3)

    img_result = img.copy()
    for i, (x1, y1, x2, y2) in enumerate(final_boxes):
        confidence = result[y1, x1]

        # Dibujar recuadro verde
        cv.rectangle(img_result, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Mostrar confianza como porcentaje fuera del recuadro
        label = f"{confidence * 100:.1f}%"
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > 20 else y1 + 30

        # Contorno blanco
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 4)

        # Texto negro
        cv.putText(img_result, label, (text_x, text_y),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        print(f"Detección {i+1} → coordenadas: ({x1},{y1}) confianza: {confidence:.2f}")

    plt.figure(figsize=(12, 8))
    plt.imshow(cv.cvtColor(img_result, cv.COLOR_BGR2RGB))
    plt.title(f"Método: TM_CCORR_NORMED con Canny\nDetecciones únicas: {len(final_boxes)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    return len(final_boxes), img_result

# Ejecutar
detections_count, result_img = detect_coca_cola_bottles('images/coca_retro_1.png', 'template/pattern.png')
print(f"Se detectaron {detections_count} botellas de Coca-Cola")


Detección 1 → coordenadas: (216,310) confianza: 0.17
Se detectaron 1 botellas de Coca-Cola


- **Mejor resultado generalizando el codigo (SIFT+FLANN+HOMOGRAPHY)** No es el que mejor nos dio en el punto 2

In [98]:
# ============================================================
# 1. CARGA DEL TEMPLATE Y PREPROCESAMIENTO
# ============================================================
template_path = './template/pattern.png'  # Ruta del template
img_template = cv.imread(template_path, 0)  # Cargar el template en escala de grises
if img_template is None:
    raise IOError("No se pudo cargar el template: " + template_path)

# Redimensionado del template
img_template = cv.resize(img_template, (0, 0), fx=0.23, fy=0.23, interpolation=cv.INTER_AREA)
# Suavizar el template para reducir ruido
img_template = cv.GaussianBlur(img_template, (3, 3), 0)
# Invertir el template -> Para obtener letras blancas
img_template = 255 - img_template.copy()

# ============================================================
# 2. CONFIGURACION DE SIFT CON PARÁMETROS AJUSTADOS
# ============================================================

sift = cv.xfeatures2d.SIFT_create(nfeatures=5000, contrastThreshold=0.01)
# nfeatures -> Máximo número de keypoints a detectar.
# contrastThreshold -> Umbral de contraste (valores menores detectan más puntos).

# ============================================================
# 3. OBTENER LA LISTA DE IMAGENES EN LA CARPETA "images"
# ============================================================
image_files = glob.glob(os.path.join('./images', '*.jpg')) + glob.glob(os.path.join('./images', '*.png'))
if not image_files:
    raise IOError("No se encontraron imágenes en la carpeta 'images'.")

# ============================================================
# 4. PROCESAR CADA IMAGEN
# ============================================================
for img_path in image_files:
    print("Procesando:", img_path)
    # Cargar la imagen destino en color y obtener su version en escala de grises
    img_color = cv.imread(img_path)
    if img_color is None:
        print("No se pudo cargar:", img_path)
        continue
    img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
    # Suavizar la imagen destino para reducir ruido
    img_proc = cv.GaussianBlur(img_gray, (3, 3), 0)
    
    # ============================================================
    # 5. EXTRAER KEYPOINTS Y DESCRIPTORES CON SIFT PARA TEMPLATE E IMAGEN DESTINO
    # ============================================================
    kp_template, des_template = sift.detectAndCompute(img_template, None)
    kp_img, des_img = sift.detectAndCompute(img_proc, None)
    print("  Keypoints - Template:", len(kp_template), "| Imagen:", len(kp_img))
    
    # ============================================================
    # 6. MATCHING CON FLANN Y RATIO TEST
    # ============================================================
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=300) 
    search_params = dict(checks=300)  
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des_template, des_img, k=2)
    # Filtradoi de buenos matches usando Ratio Test
    good_matches = [m for m, n in matches if m.distance < 0.9 * n.distance]
    print("  Good matches:", len(good_matches))
    
    # ============================================================
    # 7. CLUSTERIZACION DE LOS MATCHES CON DBSCAN
    # ============================================================
    # Coordenadas destino de cada good match
    dst_coords = np.array([kp_img[m.trainIdx].pt for m in good_matches])
    # Parámetros para DBSCAN:
    # - eps: distancia maxima (en pixeles) para considerar puntos vecinos.
    # - min_samples: minimo número de puntos para formar un cluster.
    dbscan = DBSCAN(eps=50, min_samples=1) 
    clusters = dbscan.fit_predict(dst_coords) if len(dst_coords) > 0 else np.array([])
    
    num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    print("  Número de clusters (instancias):", num_clusters)
    
    # ============================================================
    # 8. PARA CADA CLUSTER, CALCULAR HOMOGRAFIA, DIBUJAR BOUNDING BOX Y MOSTRAR CONFIDENCIA
    # ============================================================
    img_final = img_color.copy()
    unique_labels = set(clusters)
    for label in unique_labels:
        if label == -1:  # DBSCAN marca con -1 el ruido
            continue
        # indices de los matches en este cluster
        indices = np.where(clusters == label)[0]
        # Matches para poder calcular homografia
        if len(indices) < 4:
            continue
        cluster_matches = [good_matches[i] for i in indices]
        # Extraer puntos correspondientes del template y de la imagen destino
        src_pts = np.float32([kp_template[m.queryIdx].pt for m in cluster_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp_img[m.trainIdx].pt for m in cluster_matches]).reshape(-1, 1, 2)
        
        # Calcular homografia usando RANSAC. Umbral 1.0
        M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 1.0)
        if M is None:
            continue
        # Calcular nivel de confianza como la proporcion de inliers
        inliers_ratio = float(np.sum(mask)) / float(len(mask))
        
        h, w = img_template.shape  # Dimensiones del template
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst_box = cv.perspectiveTransform(pts, M)
        
        # Dibujos del bounding box sobre la imagen destino
        img_final = cv.polylines(img_final, [np.int32(dst_box)], True, (0, 255, 0), 3, cv.LINE_AA)
        # OEsquina superior
        top_left = tuple(np.int32(dst_box[0][0]))
        # NC
        cv.putText(img_final, f"{inliers_ratio*100:.1f}%", top_left, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # ============================================================
    # 9. PRINT IMAGEN FINAL
    # ============================================================
    plt.figure(figsize=(12,8))
    plt.imshow(cv.cvtColor(img_final, cv.COLOR_BGR2RGB))
    plt.title("Detección múltiple de logos con bounding boxes y confianza\n" + os.path.basename(img_path))
    plt.axis("off")
    plt.show()

Procesando: ./images\COCA-COLA-LOGO.jpg
  Keypoints - Template: 74 | Imagen: 3879
  Good matches: 26
  Número de clusters (instancias): 22
Procesando: ./images\coca_logo_1.png
  Keypoints - Template: 74 | Imagen: 652
  Good matches: 45
  Número de clusters (instancias): 2
Procesando: ./images\coca_logo_2.png
  Keypoints - Template: 74 | Imagen: 726
  Good matches: 32
  Número de clusters (instancias): 4
Procesando: ./images\coca_multi.png
  Keypoints - Template: 74 | Imagen: 5000
  Good matches: 23
  Número de clusters (instancias): 13
Procesando: ./images\coca_retro_1.png
  Keypoints - Template: 74 | Imagen: 5000
  Good matches: 11
  Número de clusters (instancias): 10
Procesando: ./images\coca_retro_2.png
  Keypoints - Template: 74 | Imagen: 1862
  Good matches: 32
  Número de clusters (instancias): 5
Procesando: ./images\logo_1.png
  Keypoints - Template: 74 | Imagen: 776
  Good matches: 28
  Número de clusters (instancias): 7
